In [ ]:
#| default_exp ciss

# Cassini ISS
> Data getters for Cassini ISS image data

In [ ]:
#| export
from yarl import URL

from planetarypy.config import config
from planetarypy.pds.opusapi import OPUS
from planetarypy.utils import have_internet, url_retrieve

In [ ]:
#| export
storage_root = config.storage_root / "missions/cassini/iss"
opus_keys = [
    "coiss_raw",
    "coiss_calib",
    "coiss_thumb",
    "coiss_medium",
    "coiss_full",
    "rms_index",
    "inventory",
    "planet_geometry",
    "ring_geometry",
    "browse_thumb",
    "browse_small",
    "browse_medium",
    "browse_full",
]

In [ ]:
storage_root

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/iss')

In [ ]:
#| export
class ISSOpus:

    def __init__(self, pid):
        self.pid = pid
        if have_internet():
            self.query_pid(pid)

    def query_pid(self, pid):
        opus = OPUS()
        self.query_result = opus.query_image_id(pid)[0]
        self.id = self.query_result[0]
        self.dict = self.query_result[1]
        for k, v in self.dict.items():
            if isinstance(v, list) and len(v) == 1:
                setattr(self, k, v[0])
            else:
                setattr(self, k, v)

    @property
    def raw_data_url(self):
        return URL(self.coiss_raw[0])

    @property
    def raw_label_url(self):
        return URL(self.coiss_raw[1])

    @property
    def raw_prefix_fmt_url(self):
        return URL(self.coiss_raw[2])

    @property
    def raw_tlmtab_url(self):
        return URL(self.coiss_raw[3])

    @property
    def calib_data_url(self):
        return URL(self.coiss_calib[0])

    @property
    def calib_label_url(self):
        return URL(self.coiss_calib[1])

    @property
    def volume(self):
        return self.raw_data_url.parts[4]

    @property
    def local_folder(self):
        return storage_root / self.pid.upper()

    @property
    def local_data_path(self):
        return self.local_folder / self.raw_data_url.name

    @property
    def local_calib_path(self):
        return self.local_folder / self.calib_data_url.name

    @property
    def local_label_path(self):
        return self.local_data_path.with_suffix(".LBL")

    @property
    def local_calib_label_path(self):
        return self.local_calib_path.with_suffix(".LBL")

    def download_raw(self, overwrite=False):
        self.local_data_path.parent.mkdir(parents=True, exist_ok=True)
        if self.local_data_path.exists() and not overwrite:
            print("File exists. Use `overwrite=True` to download fresh.")
            return
        url_retrieve(self.raw_data_url, self.local_data_path)
        url_retrieve(self.raw_label_url, self.local_label_path)

    def download_calib(self, overwrite=False):
        self.local_calib_path.parent.mkdir(parents=True, exist_ok=True)
        if self.local_calib_path.exists():
            if not overwrite:
                print("File exists. Use `overwrite` to force re-download.")
                return
        url_retrieve(self.calib_data_url, self.local_calib_path)

    def __repr__(self):
        s = f"Product ID:\n{self.id}\n\n"
        for k, v in self.query_result[1].items():
            s += f"Key: {k},\nValue(s):\n{v}\n\n"
        return s

In [ ]:
id = "N1874681996"
# id = "N1878581142"

In [ ]:
list(storage_root.rglob(f"{id}*"))

[Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/iss/N1874681996'),
 Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/iss/N1874681996/N1874681996_1_CALIB.IMG')]

In [ ]:
iss = ISSOpus(id)

Found 1 obsids.


In [ ]:
iss.local_folder

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/iss/N1874681996')

In [ ]:
iss.raw_data_url

URL('https://opus.pds-rings.seti.org/holdings/volumes/COISS_2xxx/COISS_2113/data/1874658411_1874843972/N1874681996_1.IMG')

In [ ]:
iss.download_calib()

File exists. Use `overwrite` to force re-download.


In [ ]:
config.storage_root

Path('/home/ayek72/mnt/slowdata/planetarypy')

In [ ]:
iss.dict.keys()

dict_keys(['coiss_raw', 'coiss_calib', 'coiss_thumb', 'coiss_medium', 'coiss_full', 'rms_index', 'inventory', 'planet_geometry', 'ring_geometry', 'browse_thumb', 'browse_small', 'browse_medium', 'browse_full'])

In [ ]:
iss.local_calib_path

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/iss/N1874681996/N1874681996_1_CALIB.IMG')

In [ ]:
iss.calib_data_url

URL('https://opus.pds-rings.seti.org/holdings/calibrated/COISS_2xxx/COISS_2113/data/1874658411_1874843972/N1874681996_1_CALIB.IMG')

In [ ]:
iss.volume

'COISS_2113'

In [ ]:
iss.local_folder

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/iss/N1874681996')

In [ ]:
def get_raw_data_path(product_id):
    iss = ISSOpus(product_id)
    return iss.local_data_path

In [ ]:
id

'N1874681996'

In [ ]:
list(storage_root.glob(f"**/{id}"))

[Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/iss/N1874681996')]

In [ ]:
storage_root

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/iss')

In [ ]:
!ls {storage_root}

indexes/  N1874681996/


In [ ]:
get_raw_data_path(id)

Found 1 obsids.


Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/iss/N1874681996/N1874681996_1.IMG')

In [ ]:
iss.dict

{'coiss_raw': ['https://opus.pds-rings.seti.org/holdings/volumes/COISS_2xxx/COISS_2113/data/1874658411_1874843972/N1874681996_1.IMG',
  'https://opus.pds-rings.seti.org/holdings/volumes/COISS_2xxx/COISS_2113/data/1874658411_1874843972/N1874681996_1.LBL',
  'https://opus.pds-rings.seti.org/holdings/volumes/COISS_2xxx/COISS_2113/label/prefix3.fmt',
  'https://opus.pds-rings.seti.org/holdings/volumes/COISS_2xxx/COISS_2113/label/tlmtab.fmt'],
 'coiss_calib': ['https://opus.pds-rings.seti.org/holdings/calibrated/COISS_2xxx/COISS_2113/data/1874658411_1874843972/N1874681996_1_CALIB.IMG',
  'https://opus.pds-rings.seti.org/holdings/calibrated/COISS_2xxx/COISS_2113/data/1874658411_1874843972/N1874681996_1_CALIB.LBL'],
 'coiss_thumb': ['https://opus.pds-rings.seti.org/holdings/volumes/COISS_2xxx/COISS_2113/extras/thumbnail/1874658411_1874843972/N1874681996_1.IMG.jpeg_small'],
 'coiss_medium': ['https://opus.pds-rings.seti.org/holdings/volumes/COISS_2xxx/COISS_2113/extras/browse/1874658411_187484